In [ ]:
import io
import PyPDF2
import chardet

def parse_resume(file_path):
    text = ""

    # Extract text from PDF resumes
    with open(file_path, 'rb') as file:
      raw_data = file.read()
      encoding_result = chardet.detect(raw_data)
      encoding = encoding_result['encoding']
    with open(file_path, 'r', encoding=encoding) as file:
      resume_text = file.read()

    # Extract name
    name_pattern = r"([A-Z][a-z]+)\s+([A-Z][a-z]+)"
    name_match = re.search(name_pattern, resume_text)
    name = name_match.group() if name_match else None

    # Extract email
    email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
    email_match = re.search(email_pattern, resume_text, re.I)
    email = email_match.group() if email_match else None

    # Extract phone number
    phone_pattern = r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}"
    phone_match = re.search(phone_pattern, resume_text)
    phone = phone_match.group() if phone_match else None

    # Extract skills
    skills_pattern = r"SKILLS\n+(.+?)(?:\n{2,}|$)"
    skills_match = re.search(skills_pattern, resume_text, re.DOTALL | re.IGNORECASE)
    skills = skills_match.group(1).strip() if skills_match else None

    # Extract experience
    experience_pattern = r"WORK EXPERIENCE\n+(.*?)(?:\n{2,}|$)"
    experience_match = re.search(experience_pattern, resume_text, re.DOTALL | re.IGNORECASE)
    experience = experience_match.group(1).strip() if experience_match else None

    # Extract education
    education_pattern = r"EDUCATION\n+(.*?)(?:\n{2,}|$)"
    education_match = re.search(education_pattern, resume_text, re.DOTALL | re.IGNORECASE)
    education = education_match.group(1).strip() if education_match else None

    # Return the extracted data as a dictionary
    extracted_data = {
        'name': name,
        'email': email,
        'phone': phone,
        'skills': skills,
        'experience': experience,
        'education': education
    }
    return extracted_data
# Usage
file = '/content/resume.txt'
data = parse_resume(file)
print("Name:", data.get('name'))
print("Email:", data.get('email'))
print("Phone:", data.get('phone'))
print("Skills:", data.get('skills'))
print("Experience:")
print(data.get('experience'))
print("Education:")
print(data.get('education'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: Casey Hall
Email: casey.z.hall@gmail.com
Phone: 916-566-6124
Skills: Ability to work under pressure
Financial planning
Risk assessment
Customer service
Problem solving
Technical skills
Interpersonal skills
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CERTIFICATIONS
 
Certified Financial Planner (CFP)
Certified Financial Analyst (CFA)
Experience:
BANKING ANALYST
RJ Savings and Trust
Dec 2016- Dec 2019
Managed 100 investors’ due diligence processes and coordinated with international operations managers when required.
Performed financial risk assessments and studied banking and financial market trends to prepare activity and progress reports to management.
Navigated marketing activities by reaching out to 150+ VC and lead 5% prospects.
 
Key achievement
Conducted valuation report on $20 million swaps deal for a tech company which resulted in 35% rise in corporat